In [26]:
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
pd.set_option("display.max_columns", None)

In [3]:
load_dotenv()

True

In [4]:
api_key = os.getenv("API_KEY")
db_host = os.getenv("DB_HOST")
db_password = os.getenv("DB_PASSWORD")
db_user = os.getenv("DB_USER")
db_name = os.getenv("DB_NAME")
azure_api_key = os.getenv("OPENAI_KEY")
azure_endpoint = os.getenv("OPENAI_ENDPOINT")
azureai_name = os.getenv("OPENAI_NAME")
pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [5]:
import vanna
from vanna.pinecone import PineconeDB_VectorStore
from pinecone import ServerlessSpec, Pinecone

In [6]:
from openai import AzureOpenAI
from vanna.openai import OpenAI_Chat
from vanna.remote import VannaDefault
from vanna.vannadb import VannaDB_VectorStore
from vanna.base import VannaBase

In [ ]:
#vn = VannaDefault(model=model_name, api_key=api_key)

In [7]:
index_name = 'haderach'

In [ ]:
# pc = Pinecone()

# client =pc.create_index(
#     name=index_name,
#     dimension=1024,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
# )

In [20]:
config = {
    "dimensions": 1024,
    "distance_metric": "cosine",
    "index_name": index_name,
    "api_key": pinecone_api_key,
    "fastembed_model": "BAAI/bge-large-en-v1.5"
}


In [21]:
class MyVanna(PineconeDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        PineconeDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, client=AzureOpenAI(api_version="2024-08-01-preview",
            azure_endpoint=azure_endpoint,
            api_key=azure_api_key,
           ), config={"model": azureai_name })

In [22]:
vn = MyVanna(config=config)

In [29]:
vn.connect_to_postgres(host=db_host,
                       dbname=db_name,
                       user=db_user,
                       password=db_password,
                       port=5432)

In [36]:
# df_information_schema = vn.run_sql('''SELECT *
#                                     FROM information_schema.columns
#                                     WHERE table_schema = 'public'; ''')
df_information_schema.head()

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,numeric_precision,numeric_precision_radix,numeric_scale,datetime_precision,interval_type,interval_precision,character_set_catalog,character_set_schema,character_set_name,collation_catalog,collation_schema,collation_name,domain_catalog,domain_schema,domain_name,udt_catalog,udt_schema,udt_name,scope_catalog,scope_schema,scope_name,maximum_cardinality,dtd_identifier,is_self_referencing,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,quizzard,public,Questions,Id,1,None,NO,uuid,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,quizzard,pg_catalog,uuid,None,None,None,None,1,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
1,quizzard,public,Questions,QuestionText,2,None,YES,text,NaN,1.073742e+09,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,quizzard,pg_catalog,text,None,None,None,None,2,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
2,quizzard,public,Questions,CreatedAt,3,None,NO,timestamp with time zone,NaN,NaN,NaN,NaN,NaN,6.0,None,None,None,None,None,None,None,None,None,None,None,quizzard,pg_catalog,timestamptz,None,None,None,None,3,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
3,quizzard,public,Questions,UpdatedAt,4,None,NO,timestamp with time zone,NaN,NaN,NaN,NaN,NaN,6.0,None,None,None,None,None,None,None,None,None,None,None,quizzard,pg_catalog,timestamptz,None,None,None,None,4,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
4,quizzard,public,Questions,QuizId,5,None,NO,uuid,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,quizzard,pg_catalog,uuid,None,None,None,None,5,NO,NO,None,None,None,None,None,NO,NEVER,None,YES


In [ ]:
plan = vn.get_training_plan_generic(df_information_schema)

Train on Information Schema: quizzard.public Questions
Train on Information Schema: quizzard.public QuizSessions
Train on Information Schema: quizzard.public QuizCategories
Train on Information Schema: quizzard.public Quizzes
Train on Information Schema: quizzard.public __EFMigrationsHistory
Train on Information Schema: quizzard.public Users
Train on Information Schema: quizzard.public Options
Train on Information Schema: quizzard.public Answers

In [32]:
for table_name in plan.get_summary():
  print(table_name[len("Train on Information Schema: "):])

quizzard.public Questions
quizzard.public QuizSessions
quizzard.public QuizCategories
quizzard.public Quizzes
quizzard.public __EFMigrationsHistory
quizzard.public Users
quizzard.public Options
quizzard.public Answers


In [37]:
df_information_schema.head()

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,numeric_precision,numeric_precision_radix,numeric_scale,datetime_precision,interval_type,interval_precision,character_set_catalog,character_set_schema,character_set_name,collation_catalog,collation_schema,collation_name,domain_catalog,domain_schema,domain_name,udt_catalog,udt_schema,udt_name,scope_catalog,scope_schema,scope_name,maximum_cardinality,dtd_identifier,is_self_referencing,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,quizzard,public,Questions,Id,1,None,NO,uuid,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,quizzard,pg_catalog,uuid,None,None,None,None,1,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
1,quizzard,public,Questions,QuestionText,2,None,YES,text,NaN,1.073742e+09,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,quizzard,pg_catalog,text,None,None,None,None,2,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
2,quizzard,public,Questions,CreatedAt,3,None,NO,timestamp with time zone,NaN,NaN,NaN,NaN,NaN,6.0,None,None,None,None,None,None,None,None,None,None,None,quizzard,pg_catalog,timestamptz,None,None,None,None,3,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
3,quizzard,public,Questions,UpdatedAt,4,None,NO,timestamp with time zone,NaN,NaN,NaN,NaN,NaN,6.0,None,None,None,None,None,None,None,None,None,None,None,quizzard,pg_catalog,timestamptz,None,None,None,None,4,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
4,quizzard,public,Questions,QuizId,5,None,NO,uuid,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,quizzard,pg_catalog,uuid,None,None,None,None,5,NO,NO,None,None,None,None,None,NO,NEVER,None,YES


In [ ]:
vn.train(plan=plan)

In [ ]:
# DDL = '''
#     CREATE TABLE IF NOT EXISTS public."Answers" (
#         "Id" uuid NOT NULL DEFAULT gen_random_uuid(),
#         "CreatedAt" timestamptz NOT NULL,
#         "LastUpdatedAt" timestamptz NOT NULL,
#         "OptionId" uuid NOT NULL,
#         "QuestionId" uuid NOT NULL,
#         "QuizSessionId" uuid NOT NULL,
#         "UserId" uuid NOT NULL,
#         CONSTRAINT "Answers_pkey" PRIMARY KEY ("Id"),
#         CONSTRAINT "Answers_OptionId_fkey" FOREIGN KEY ("OptionId") 
#             REFERENCES public."Options" ("Id") ON DELETE CASCADE,
#         CONSTRAINT "Answers_QuestionId_fkey" FOREIGN KEY ("QuestionId") 
#             REFERENCES public."Questions" ("Id") ON DELETE CASCADE,
#         CONSTRAINT "Answers_QuizSessionId_fkey" FOREIGN KEY ("QuizSessionId") 
#             REFERENCES public."QuizSessions" ("Id") ON DELETE CASCADE,
#         CONSTRAINT "Answers_UserId_fkey" FOREIGN KEY ("UserId") 
#             REFERENCES public."Users" ("Id") ON DELETE CASCADE
#     );
# '''

In [24]:
DDL = '''
CREATE TABLE IF NOT EXISTS public."Answers" (
    "Id" uuid PRIMARY KEY DEFAULT gen_random_uuid(),
    "CreatedAt" timestamptz NOT NULL,
    "LastUpdatedAt" timestamptz NOT NULL,
    "OptionId" uuid NOT NULL REFERENCES public."Options" ("Id") ON DELETE CASCADE,
    "QuestionId" uuid NOT NULL REFERENCES public."Questions" ("Id") ON DELETE CASCADE,
    "QuizSessionId" uuid NOT NULL REFERENCES public."QuizSessions" ("Id") ON DELETE CASCADE,
    "UserId" uuid NOT NULL REFERENCES public."Users" ("Id") ON DELETE CASCADE
);
'''

In [25]:
vn.train(ddl=DDL)

Adding ddl: 
CREATE TABLE IF NOT EXISTS public."Answers" (
    "Id" uuid PRIMARY KEY DEFAULT gen_random_uuid(),
    "CreatedAt" timestamptz NOT NULL,
    "LastUpdatedAt" timestamptz NOT NULL,
    "OptionId" uuid NOT NULL REFERENCES public."Options" ("Id") ON DELETE CASCADE,
    "QuestionId" uuid NOT NULL REFERENCES public."Questions" ("Id") ON DELETE CASCADE,
    "QuizSessionId" uuid NOT NULL REFERENCES public."QuizSessions" ("Id") ON DELETE CASCADE,
    "UserId" uuid NOT NULL REFERENCES public."Users" ("Id") ON DELETE CASCADE
);



'353813-ddl'

In [ ]:
vn.train(documentation="When querying any table, use the fully qualified table name including schema")

In [27]:
sql_queries = [
    ''' SELECT 
    "QuizSessionId",
    COUNT(*) AS answers_in_session,
    MIN("CreatedAt") AS session_start,
    MAX("CreatedAt") AS session_end,
    COUNT(DISTINCT "UserId") AS unique_users
    FROM public."Answers"
    GROUP BY "QuizSessionId"
    ORDER BY session_start;
    ''',

    ''' SELECT 
    q."QuestionText",
    COUNT(a.*) AS times_answered,
    COUNT(DISTINCT a."UserId") AS unique_users_answered
    FROM public."Answers" a
    JOIN public."Questions" q ON q."Id" = a."QuestionId"
    GROUP BY a."QuestionId", q."QuestionText"
    ORDER BY times_answered DESC;
    ''',

    ''' SELECT 
    CONCAT(u."FirstName", ' ', u."LastName") AS FullName, 
    COUNT(q."Score") AS points,
    TO_CHAR(q."EndTime" - q."StartTime", 'HH24:MI:SS') AS duration
    FROM public."Users" AS u
    JOIN public."QuizSessions" AS q ON u."Id" = q."UserId"
    GROUP BY FullName, duration
    ORDER BY points DESC;
    ''',

    ''' SELECT 
    CONCAT("FirstName", ' ', "LastName") AS FullName,
    "Email"
    FROM public."Users"
    WHERE "EmailConfirmed" = true;
    '''
]

In [ ]:
vn.run_sql(sql_queries[1]).head()

In [ ]:
vn.run_sql(sql_queries[3]).head()

In [11]:
vn.train(
        question="What are the most popular questions being answered, and how many unique users have answered each one?",
        sql=sql_queries[1]
)

'0d9b503a9fd7e3f2d57d1786c462003b-sql'

In [33]:
vn.train(sql=sql_queries[2])

Using model gpt-35-turbo for 132.75 tokens (approx)
Question generated with sql: What is the full name, total points, and duration of each user's quiz session, sorted by the highest number of points? 
Adding SQL...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

'e6b70182-6f27-56ca-9391-1d3be5b38806-sql'

In [33]:
vn.train(question= "Who are the users with confirmed emails?",
         sql=sql_queries[3])

'b3a1d2b6f561005f98928c55071b2528-sql'

In [ ]:
answer = vn.ask(question="What are the most popular questions being answered, and how many unique users have answered each one?", allow_llm_to_see_data=True)

print(answer)

In [ ]:
vn.get_training_data()

In [37]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn)
app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Using model gpt-35-turbo for 2804.25 tokens (approx)
